In [15]:
import win32com.client
import os
import shutil
import zipfile
import patoolib
from pathlib import Path
from datetime import date

## download email attachments

In [16]:
def download_unread_attachments(download_folder, target_account_email=None):
    
    # Create download folder if it doesn't exist
    Path(download_folder).mkdir(parents=True, exist_ok=True)
    
    # Connect to Outlook
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    
    # If you have multiple accounts and want to target your Gmail specifically
    if target_account_email:
        inbox = None
        for folder in outlook.Folders:
            # Find the folder matching your Gmail account
            if target_account_email.lower() in folder.Name.lower():
                # Navigate to Inbox within that account
                for subfolder in folder.Folders:
                    if subfolder.Name == "Inbox":
                        inbox = subfolder
                        break
                break
        
        if inbox is None:
            print(f"Could not find inbox for {target_account_email}")
            return
    else:
        # Use default inbox (if you only have one account)
        inbox = outlook.GetDefaultFolder(6)  # 6 = Inbox folder
    
    # Get all messages
    messages = inbox.Items
    
    # Filter for unread messages only
    unread_messages = messages.Restrict("[Unread] = True")
    
    print(f"Found {len(unread_messages)} unread messages")
    
    attachment_count = 0
    
    # Loop through unread messages
    for message in unread_messages:
        try:
            # Check if message has attachments
            if message.Attachments.Count > 0:
                print(f"\nProcessing: {message.Subject}")
                print(f"  From: {message.SenderEmailAddress}")
                print(f"  Attachments: {message.Attachments.Count}")
                
                # Download each attachment
                for attachment in message.Attachments:
                    filename = attachment.FileName
                    filepath = os.path.join(download_folder, filename)
                    
                    # Handle duplicate filenames
                    counter = 1
                    base_name, extension = os.path.splitext(filename)
                    while os.path.exists(filepath):
                        filename = f"{base_name}_{counter}{extension}"
                        filepath = os.path.join(download_folder, filename)
                        counter += 1
                    
                    # Save attachment
                    attachment.SaveAsFile(filepath)
                    print(f"  ✓ Saved: {filename}")
                    attachment_count += 1
                message.Unread = True
                
        except Exception as e:
            print(f"Error processing message: {str(e)}")
            continue
    
    print(f"\n{'='*50}")
    print(f"Download complete! Total attachments saved: {attachment_count}")
    print(f"Location: {download_folder}")

## extract mdb files

In [ ]:
def process_archives(source_dir, final_mdb_dir):
    """
    1. Finds all .zip and .rar files in source_dir
    2. Extracts them to a temporary folder
    3. Moves any .mdb files found to final_mdb_dir
    4. Cleans up temp files
    """
    
    # 1. Setup paths
    source_path = Path(source_dir)
    final_path = Path(final_mdb_dir)
    final_path.mkdir(parents=True, exist_ok=True)
    
    # Temporary folder for extraction
    temp_extract_path = source_path / "temp_extracted"
    
    # 2. Iterate through all files in directory
    for file_path in source_path.glob("*"):
        
        # Skip if it's a directory
        if file_path.is_dir():
            continue
            
        extraction_success = False
        current_temp_folder = temp_extract_path / file_path.stem
        
        try:
            # === HANDLE ZIP FILES ===
            if file_path.suffix.lower() == ".zip":
                print(f"Extracting ZIP: {file_path.name}")
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(current_temp_folder)
                extraction_success = True
                
            # === HANDLE RAR FILES ===
            elif file_path.suffix.lower() == ".rar":
                print(f"Extracting RAR: {file_path.name}")
                # Create temp folder first as patool needs it to exist
                current_temp_folder.mkdir(parents=True, exist_ok=True)
                patoolib.extract_archive(str(file_path), outdir=str(current_temp_folder))
                extraction_success = True
                
        except Exception as e:
            print(f"Error extracting {file_path.name}: {e}")
            continue

        # 3. If extraction worked, find and move .mdb files
        if extraction_success:
            # Walk through the extracted folder (recursively) to find MDBs
            for root, dirs, files in os.walk(current_temp_folder):
                for file in files:
                    if file.lower().endswith(".mdb"):
                        
                        # Source path of the mdb
                        mdb_source = Path(root) / file
                        
                        # Destination path (handle duplicates)
                        mdb_dest = final_path / file
                        counter = 1
                        while mdb_dest.exists():
                            mdb_dest = final_path / f"{mdb_source.stem}_{counter}{mdb_source.suffix}"
                            counter += 1
                            
                        # Move the file
                        shutil.move(str(mdb_source), str(mdb_dest))
                        print(f"  -> Found and moved: {mdb_dest.name}")

            # Cleanup: Remove the temp folder for this archive
            shutil.rmtree(current_temp_folder)

    # Final Cleanup: Remove the main temp directory if empty
    if temp_extract_path.exists():
        try:
            temp_extract_path.rmdir() 
        except:
            pass # Ignore if not empty (shouldn't happen)

    print(f"\nProcessing Complete. All MDB files are in: {final_path}")

# ==========================================
# USAGE
# ==========================================
SOURCE_DIRECTORY = r"C:\Data\Archives"       # Folder containing zip/rar files
MDB_DESTINATION  = r"C:\Data\Final_MDBs"     # Folder where MDBs will go

process_archives(SOURCE_DIRECTORY, MDB_DESTINATION)

In [18]:
if __name__ == "__main__":

    today = date.today().strftime("%Y.%m.%d")
    
    # Define your download folder
    DOWNLOAD_FOLDER = f"F:/Downloads/{today}"
    
    # If you have multiple accounts, specify your Gmail address
    # Leave as None if you only have one account
    GMAIL_ADDRESS = "tamilnadu.aswing@gmail.com"
    
    # Run the function
    download_unread_attachments(DOWNLOAD_FOLDER, GMAIL_ADDRESS)

Found 3 unread messages

Processing: कृषि सांख्यिकी अनुसूची 2.0, प्रतिदर्श-केन्द्रीय, मौसम-खरीफ, वर्ष 2025-26 के संबंध मे
  From: as2.0fbdhqrs@gmail.com
  Attachments: 6
  ✓ Saved: F133380291002_26Nov2025_C1TN38PADDY_KAR0222025-26.jpeg
  ✓ Saved: F133380291001_26Nov2025_C1TN38PADDY_KAR0212025-26.jpeg
  ✓ Saved: F133380291002_26Nov2025_C1TN38PADDY_KAR0222025-26.zip
  ✓ Saved: F133380291001_26Nov2025_C1TN38PADDY_KAR0212025-26.zip
  ✓ Saved: P_EXPTID_133380291001_C1TN38PADDY_KAR0212025-26_26Nov2025.docx
  ✓ Saved: P_EXPTID_133380291002_C1TN38PADDY_KAR0222025-26_26Nov2025.docx

Processing: कृषि सांख्यिकी अनुसूची 2.0, प्रतिदर्श-केन्द्रीय, मौसम-खरीफ, वर्ष 2025-26 के संबंध मे
  From: as2.0fbdhqrs@gmail.com
  Attachments: 2
  ✓ Saved: F133010291001_23Sep2025_C1TN01PADDY_KAR0212025-26.zip
  ✓ Saved: F133010291002_25Sep2025_C1TN01PADDY_KAR0222025-26.zip

Processing: कृषि सांख्यिकी अनुसूची 2.0, प्रतिदर्श-केन्द्रीय, मौसम-खरीफ, वर्ष 2025-26 के संबंध मे
  From: as2.0fbdhqrs@gmail.com
  Attachments: 